In [9]:
import wetter_api as wt
# import waldbrand
import model_eval as me
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error, r2_score

In [47]:
parameter_list = [
    wt.DwdObservationParameter.MONTHLY.PRECIPITATION_HEIGHT,
    wt.DwdObservationParameter.MONTHLY.CLIMATE_SUMMARY.WIND_FORCE_BEAUFORT,
    wt.DwdObservationParameter.MONTHLY.TEMPERATURE_AIR_MAX_200,
]
# Dictionary with short names as keys and full names as values
shortname_to_state = {
    "BW": "Baden-Württemberg",
    "BY": "Bayern",
    "BE": "Berlin",
    "BB": "Brandenburg",
    "HB": "Bremen",
    "HH": "Hamburg",
    "HE": "Hessen",
    "MV": "Mecklenburg-Vorpommern",
    "NI": "Niedersachsen",
    "NW": "Nordrhein-Westfalen",
    "RP": "Rheinland-Pfalz",
    "SL": "Saarland",
    "SN": "Sachsen",
    "ST": "Sachsen-Anhalt",
    "SH": "Schleswig-Holstein",
    "TH": "Thüringen",
}

# Example usage
shortname = "BB"
state = shortname_to_state.get(shortname)

In [ ]:
# shortname = "BW"
# state = shortname_to_state.get(shortname)
#merged_data = me.prepare_data(parameter_list, state)
import pandas as pd
import common_paths
merged_data = pd.read_csv(common_paths.DATA.joinpath("dwd/Brandenburg.csv"))

In [ ]:
df_past = merged_data.where(merged_data[[]])
df_future

In [49]:
test_size=0.2
random_state=42
str_parameter_list = [str(param) for param in parameter_list]
feature_list = ["Year", "Month"] + str_parameter_list
X = merged_data[feature_list]
y = merged_data["nFires"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

print(y_train)
# results = evaluate_model(X_train, X_test, y_train, y_test)

31900     0.0
67535     3.0
187469    0.0
29699     0.0
39303     4.0
         ... 
141744    0.0
122606    0.0
156580    0.0
174331    0.0
144768    1.0
Name: nFires, Length: 146182, dtype: float64


In [51]:
# How to define this?
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": list(range(5, 21, 5)),  # Smaller range for max depth
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 5, 10]
}
rf = RandomForestRegressor()

grid_search = GridSearchCV(
    estimator=rf, param_grid=param_grid, cv=10, scoring="neg_mean_squared_error"
)

#searching for the right parameters using random Forest on a 10-fold Cross Validation (cv) and the nmse
grid_search.fit(X_train, y_train)

# saving best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

KeyboardInterrupt: 

In [44]:
print(best_params)
print(best_model)
#grid_search.cv_results_
#grid_search.best_index_

{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
RandomForestRegressor(max_depth=20, min_samples_split=5)


In [45]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Evaluate the best model

#predict 'Anzahl Waldbrände' with the model
y_pred = best_model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2 Score: {r2}")
print(f"MAE: {mae}")

MSE: 2.1759962049335927e-06
R2 Score: 0.9988510698669202
MAE: 0.00011100569259962064
